In [ ]:
# get train data
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()
wb = webdriver.Chrome()

for i in range(23376, 100000):
    try:
        print(f"Step {i}\n")
        wb.get(f"https://writing.stackexchange.com/questions/{i}")
        time.sleep(2)
        open("train_data.txt", "a").write("question: "+wb.find_element(By.CLASS_NAME, 'question-hyperlink').text+"\n"+("\n".join(wb.find_element(By.ID, 'question').text.split("\n")[2:])).split("\nShare\n")[0]+"\nanswer: "+wb.find_element(By.CLASS_NAME, 'answercell').text.split("\nShare\n")[0]+"\n\n")
    except:
        pass


In [2]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm

def fine_tune_model(model, tokenizer, train_data_chunk):
    inputs = tokenizer(train_data_chunk, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"]
    if input_ids.numel() == 0:
        return  # Skip empty input chunks
    input_shape = input_ids.shape
    if len(input_shape) > 1:
        input_ids = input_ids.view(-1, input_shape[-1])
    else:
        input_ids = input_ids.unsqueeze(0)  # Add a batch dimension if input is 1D
    outputs = model(input_ids=input_ids, labels=input_ids)
    loss = outputs.loss
    loss.backward()

# Load tokenizer and model
print("Loading tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
custom_config = GPT2Config(
    vocab_size=50257,  # Assuming you're using the default GPT-2 tokenizer's vocabulary size
    n_positions=512,  # Maximum sequence length
    n_ctx=512,  # Context size
    n_embd=768,  # Dimensionality of the embeddings and hidden states
    n_layer=128,  # Number of layers
    n_head=24,  # Number of attention heads
)

model = GPT2LMHeadModel(custom_config)

# Load training data
print("Loading training data...")
train_data_path = "train_data.txt"
train_data = open(train_data_path, 'r', encoding="utf-8", errors='ignore').read().strip().split("\n\n")
train_data_lines = len(train_data)

# Preprocess data and fine-tune the model
print("Preprocessing data and fine-tuning model...")
tokenizer.pad_token = tokenizer.eos_token
progress_bar = tqdm(total=train_data_lines, desc="Fine-tuning Progress")
for chunk in train_data:
    fine_tune_model(model, tokenizer, chunk)
    progress_bar.update(1)

# Optionally, save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2")

print("Fine-tuning completed and model saved.")


Loading tokenizer...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:31                                                                                   │
│                                                                                                  │
│   28 │   n_head=24,  # Number of attention heads                                                 │
│   29 )                                                                                           │
│   30                                                                                             │
│ ❱ 31 model = GPT2LMHeadModel(custom_config)                                                      │
│   32                                                                                             │
│   33 # Load training data                                                                        │
│   34 print("Loading training data...")                                                           │
│                                                                                                  │
│ C:\Users\Ivan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCach │
│ e\local-packages\Python37\site-packages\transformers\models\gpt2\modeling_gpt2.py:965 in         │
│ __init__                                                                                         │
│                                                                                                  │
│    962 │                                                                                         │
│    963 │   def __init__(self, config):                                                           │
│    964 │   │   super().__init__(config)                                                          │
│ ❱  965 │   │   self.transformer = GPT2Model(config)                                              │
│    966 │   │   self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)            │
│    967 │   │                                                                                     │
│    968 │   │   # Model parallel                                                                  │
│                                                                                                  │
│ C:\Users\Ivan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCach │
│ e\local-packages\Python37\site-packages\transformers\models\gpt2\modeling_gpt2.py:683 in         │
│ __init__                                                                                         │
│                                                                                                  │
│    680 │   │   self.wpe = nn.Embedding(config.max_position_embeddings, self.embed_dim)           │
│    681 │   │                                                                                     │
│    682 │   │   self.drop = nn.Dropout(config.embd_pdrop)                                         │
│ ❱  683 │   │   self.h = nn.ModuleList([GPT2Block(config, layer_idx=i) for i in range(config.num  │
│    684 │   │   self.ln_f = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_epsilon)           │
│    685 │   │                                                                                     │
│    686 │   │   # Model parallel                                                                  │
│                                                                                                  │
│ C:\Users\Ivan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCach │
│ e\local-packages\Python37\site-packages\transformers\models\gpt2\modeling_gpt2.py:683 in         │
│ <listcomp>                                                                                       │
│                                                                                                  │
│    680 │   │   self.wpe = nn.Embedding(config.max_position_

In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("fine_tuned_gpt2")

def ask_question(question):
    input_ids = tokenizer.encode(question, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=200, num_return_sequences=1)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

question = ""
while question != "exit":
    question = input("Q: ")
    answer = ask_question(question)
    print("A:\n"+answer)

Q:  how are you


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A:
how are you going to do it?"

"I'm going to do it," she said. "I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going to do it. I'm going


Q:  can you give me a python code example about requests?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A:
What is your name?

I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a long time. I'm a guy who's been doing this for a


Q:  你知道我是誰嗎？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A:
你知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是誰嗎？。

知道我是
